<a href="https://colab.research.google.com/github/Gou-k/colab-github-demo-mokumokukai/blob/main/20251026_09_API%E3%82%AD%E3%83%83%E3%82%AF%E3%82%AA%E3%83%95_%26_%E6%9C%80%E5%B0%8F%E3%83%97%E3%83%AD%E3%83%80%E3%82%AF%E3%83%88%E3%82%92%E4%BD%9C%E3%82%8B_AI%E3%82%A2%E3%83%89%E3%83%90%E3%82%A4%E3%82%B6%E3%83%BC%EF%BC%88%E5%B0%8F%E6%9E%97%E5%89%9B%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai

In [ ]:
# @title
import os
import json
import sys
from openai import OpenAI
from google.colab import userdata

# 文字コード対応（Windows/Mac/Linux）
if sys.platform == 'win32':
    sys.stdout.reconfigure(encoding='utf-8')
elif sys.platform == 'darwin':
    sys.stdout.reconfigure(encoding='utf-8')

# OpenAI APIキーの設定
client = OpenAI(api_key=userdata.get('openai_api_key2'))


def analyze_emotion_pad(text):
    """
    Responses APIを使ってPADモデルで感情分析を行う

    Args:
        text (str): 分析対象のテキスト

    Returns:
        dict: PAD値 {"pleasure": float, "arousal": float, "dominance": float}
    """
    prompt = f"""あなたは感情分析の専門家です。以下の文章をPADモデル(Pleasure-Arousal-Dominance model)で感情分析してください。

文章：{text}

必ず以下のJSON形式**のみ**で回答してください（他の文章は一切含めないでください）：
{{
  "pleasure": 0.0,
  "arousal": 0.0,
  "dominance": 0.0
}}

各値は-1.0から1.0の範囲で指定してください。
- pleasure: 快-不快の度合い（正の値=快、負の値=不快）
- arousal: 興奮-沈静の度合い（正の値=興奮、負の値=沈静）
- dominance: 支配-服従の度合い（正の値=支配的、負の値=服従的）

JSON形式のみで回答してください。"""

    try:
        # Responses API を使用
        response = client.responses.create(
            model="gpt-4o-mini",
            input=prompt,
            store=False
        )

        # output_textから結果を取得してJSONパース
        output = response.output_text.strip()

        # JSONブロックから抽出（```json ``` で囲まれている場合に対応）
        if "```json" in output:
            output = output.split("```json")[1].split("```")[0].strip()
        elif "```" in output:
            output = output.split("```")[1].split("```")[0].strip()

        pad_values = json.loads(output)
        return pad_values
    except json.JSONDecodeError as e:
        print(f"JSON解析エラー: {e}")
        print(f"受信したテキスト: {output}")
        return {"pleasure": 0.0, "arousal": 0.0, "dominance": 0.0}
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return {"pleasure": 0.0, "arousal": 0.0, "dominance": 0.0}


def calculate_emotions(pad):
    """
    PAD値から6つの基本感情スコアを計算する

    Args:
        pad (dict): PAD値

    Returns:
        dict: 6つの感情スコア（0-100%）
    """
    p = pad["pleasure"]
    a = pad["arousal"]
    d = pad["dominance"]

    # PAD値を0-1の範囲に正規化
    def normalize(value):
        return (value + 1.0) / 2.0

    emotions = {}

    # 喜び: 快 × 興奮 × 支配
    if p > 0 and a > 0 and d > 0:
        emotions["喜び　"] = int(normalize(p) * normalize(a) * normalize(d) * 100)
    else:
        emotions["喜び　"] = 0

    # 怒り: 不快 × 興奮 × 支配
    if p < 0 and a > 0 and d > 0:
        emotions["怒り　"] = int(normalize(-p) * normalize(a) * normalize(d) * 100)
    else:
        emotions["怒り　"] = 0

    # 恐れ: 不快 × 興奮 × 服従
    if p < 0 and a > 0 and d < 0:
        emotions["恐れ　"] = int(normalize(-p) * normalize(a) * normalize(-d) * 100)
    else:
        emotions["恐れ　"] = 0

    # 悲しみ: 不快 × 沈静
    if p < 0 and a < 0:
        emotions["悲しみ"] = int(normalize(-p) * normalize(-a) * 100)
    else:
        emotions["悲しみ"] = 0

    # 安心: 快 × 沈静
    if p > 0 and a < 0:
        emotions["安心　"] = int(normalize(p) * normalize(-a) * 100)
    else:
        emotions["安心　"] = 0

    # 驚き: 興奮の絶対値
    emotions["驚き　"] = int(normalize(abs(a)) * 100)

    return emotions


def draw_progress_bar(percentage, width=20):
    """
    プログレスバーを描画する

    Args:
        percentage (int): パーセンテージ（0-100）
        width (int): バーの幅

    Returns:
        str: プログレスバーの文字列
    """
    filled = int(width * percentage / 100)
    bar = "█" * filled + "░" * (width - filled)
    return bar


def generate_message(emotions):
    """
    Responses APIを使って感情スコアから親しみやすいメッセージを生成する

    Args:
        emotions (dict): 感情スコア

    Returns:
        str: メッセージ
    """
    emotions_text = "\n".join([f"- {key}: {value}%" for key, value in emotions.items()])

    prompt = f"""以下の感情分析結果を見て、専門用語を使わず、やさしく親しみやすい言葉でユーザーに説明してください。

感情スコア：
{emotions_text}

以下の条件を守ってください：
- 100文字程度で簡潔に
- 共感的で温かい口調
- 専門用語は使わない
- ユーザーの気持ちに寄り添う表現"""

    try:
        # Responses API を使用
        response = client.responses.create(
            model="gpt-4o-mini",
            input=prompt,
            store=False
        )

        message_content = response.output_text

        if message_content is None or message_content.strip() == "":
            return "お疲れさまでした。少し休んでくださいね。"

        return message_content.strip()
    except Exception as e:
        print(f"メッセージ生成エラー: {e}")
        return "あなたの気持ちを分析しました。今日も一日お疲れさまでした。"


def main():
    """メイン処理"""
    print("=" * 50)
    print("  📔 今日の気持ち日記")
    print("=" * 50)
    print("  🆕 OpenAI Responses API 使用")
    print("=" * 50)
    print()

    while True:
        # ユーザー入力
        user_input = input("今日の出来事を1行で教えてください：\n> ")

        if not user_input.strip():
            print("何か入力してください。")
            continue

        print("\n分析中...\n")

        # PAD分析（Responses API使用）
        pad_values = analyze_emotion_pad(user_input)

        # 6感情スコア計算
        emotions = calculate_emotions(pad_values)

        # 結果表示
        print("【感情スコア】")
        emotion_icons = {
            "喜び　": "😊",
            "怒り　": "😠",
            "恐れ　": "😨",
            "悲しみ": "😢",
            "安心　": "😌",
            "驚き　": "😲"
        }

        for emotion_name in ["喜び　", "怒り　", "恐れ　", "悲しみ", "安心　", "驚き　"]:
            icon = emotion_icons[emotion_name]
            score = emotions[emotion_name]
            bar = draw_progress_bar(score)
            padding = " " * (4 - len(emotion_name))
            print(f"{icon} {emotion_name}{padding} {bar} {score:3d}%")

        # AIメッセージ生成（Responses API使用）
        print("\n【AIからのメッセージ】")
        message = generate_message(emotions)
        print(message)
        print()

        # 継続確認
        continue_input = input("もう一度分析しますか？ (y/n): ")
        if continue_input.lower() != 'y':
            print("\nありがとうございました！またお会いしましょう 😊")
            break
        print()


# APIキーが設定されているか確認
try:
    test_key = userdata.get('openai_api_key2')
    if test_key:
        main()
    else:
        print("エラー: APIキーが設定されていません。")
        print("Google Colab Secretsに 'openai_api_key2' を設定してください。")
except Exception as e:
    print(f"エラー: {e}")
    print("Google Colab Secretsに 'openai_api_key2' を設定してください。")

  📔 今日の気持ち日記
  🆕 OpenAI Responses API 使用

今日の出来事を1行で教えてください：
> aaaaaaaaaa

分析中...

【感情スコア】
😊 喜び　  ░░░░░░░░░░░░░░░░░░░░   0%
😠 怒り　  ░░░░░░░░░░░░░░░░░░░░   0%
😨 恐れ　  ░░░░░░░░░░░░░░░░░░░░   0%
😢 悲しみ  ░░░░░░░░░░░░░░░░░░░░   0%
😌 安心　  ░░░░░░░░░░░░░░░░░░░░   0%
😲 驚き　  ██████████░░░░░░░░░░  50%

【AIからのメッセージ】
あなたの気持ちは驚きが大きいみたいですね！何か新しいことに出会ったのかな？その感情があるのは、特別な瞬間だから。少し休んで、自分の気持ちを大事にしてくださいね。

もう一度分析しますか？ (y/n): n

ありがとうございました！またお会いしましょう 😊


In [ ]:
# @title
import os
import json
import sys
from openai import OpenAI
from google.colab import userdata

# 文字コード対応（Windows/Mac/Linux）
if sys.platform == 'win32':
    sys.stdout.reconfigure(encoding='utf-8')
elif sys.platform == 'darwin':
    sys.stdout.reconfigure(encoding='utf-8')

# Google Colab Secretsから環境変数に設定
os.environ["openai_api_key"] = userdata.get('openai_api_key2')

# OpenAI APIキーの設定（直接userdataから取得）
client = OpenAI(api_key=userdata.get('openai_api_key2'))

# Responses APIのresponse_idを保持するグローバル変数
current_response_id = None


def analyze_emotion_pad(text):
    """
    Responses APIを使ってPADモデルで感情分析を行う

    Args:
        text (str): 分析対象のテキスト

    Returns:
        dict: PAD値 {"pleasure": float, "arousal": float, "dominance": float}
    """
    prompt = f"""あなたは感情分析の専門家です。以下の文章をPADモデル(Pleasure-Arousal-Dominance model)で感情分析してください。

文章：{text}

必ず以下のJSON形式**のみ**で回答してください（他の文章は一切含めないでください）：
{{
  "pleasure": 0.0,
  "arousal": 0.0,
  "dominance": 0.0
}}

各値は-1.0から1.0の範囲で指定してください。
- pleasure: 快-不快の度合い（正の値=快、負の値=不快）
- arousal: 興奮-沈静の度合い（正の値=興奮、負の値=沈静）
- dominance: 支配-服従の度合い（正の値=支配的、負の値=服従的）

JSON形式のみで回答してください。"""

    try:
        # Responses API を使用
        response = client.responses.create(
            model="gpt-4o-mini",  # 利用可能なモデル名に変更
            input=prompt,
            store=False  # 会話履歴を保存しない
        )

        # output_textから結果を取得してJSONパース
        output = response.output_text.strip()

        # JSONブロックから抽出（```json ``` で囲まれている場合に対応）
        if "```json" in output:
            output = output.split("```json")[1].split("```")[0].strip()
        elif "```" in output:
            output = output.split("```")[1].split("```")[0].strip()

        pad_values = json.loads(output)
        return pad_values
    except json.JSONDecodeError as e:
        print(f"JSON解析エラー: {e}")
        print(f"受信したテキスト: {output}")
        return {"pleasure": 0.0, "arousal": 0.0, "dominance": 0.0}
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return {"pleasure": 0.0, "arousal": 0.0, "dominance": 0.0}


def calculate_emotions(pad):
    """
    PAD値から6つの基本感情スコアを計算する（改善版）

    Args:
        pad (dict): PAD値

    Returns:
        dict: 6つの感情スコア（0-100%）
    """
    p = pad["pleasure"]
    a = pad["arousal"]
    d = pad["dominance"]

    # PAD値を0-1の範囲に正規化
    def normalize(value):
        return (value + 1.0) / 2.0

    emotions = {}

    # 喜び: 快 × 興奮 × 支配
    if p > 0 and a > 0 and d > 0:
        emotions["喜び　"] = int(normalize(p) * normalize(a) * normalize(d) * 100)
    else:
        emotions["喜び　"] = 0

    # 怒り: 不快 × 興奮 × 支配
    if p < 0 and a > 0 and d > 0:
        emotions["怒り　"] = int(normalize(-p) * normalize(a) * normalize(d) * 100)
    else:
        emotions["怒り　"] = 0

    # 恐れ: 不快 × 興奮 × 服従
    if p < 0 and a > 0 and d < 0:
        emotions["恐れ　"] = int(normalize(-p) * normalize(a) * normalize(-d) * 100)
    else:
        emotions["恐れ　"] = 0

    # 悲しみ: 不快 × 沈静
    if p < 0 and a < 0:
        emotions["悲しみ"] = int(normalize(-p) * normalize(-a) * 100)
    else:
        emotions["悲しみ"] = 0

    # 安心: 快 × 沈静
    if p > 0 and a < 0:
        emotions["安心　"] = int(normalize(p) * normalize(-a) * 100)
    else:
        emotions["安心　"] = 0

    # 驚き: 興奮の絶対値
    emotions["驚き　"] = int(normalize(abs(a)) * 100)

    return emotions


def draw_progress_bar(percentage, width=20):
    """
    プログレスバーを描画する

    Args:
        percentage (int): パーセンテージ（0-100）
        width (int): バーの幅

    Returns:
        str: プログレスバーの文字列
    """
    filled = int(width * percentage / 100)
    bar = "█" * filled + "░" * (width - filled)
    return bar


def generate_message(emotions):
    """
    Responses APIを使って感情スコアから親しみやすいメッセージを生成する

    Args:
        emotions (dict): 感情スコア

    Returns:
        str: メッセージ
    """
    emotions_text = "\n".join([f"- {key}: {value}%" for key, value in emotions.items()])

    prompt = f"""以下の感情分析結果を見て、専門用語を使わず、やさしく親しみやすい言葉でユーザーに説明してください。

感情スコア：
{emotions_text}

以下の条件を守ってください：
- 100文字程度で簡潔に
- 共感的で温かい口調
- 専門用語は使わない
- ユーザーの気持ちに寄り添う表現"""

    try:
        # Responses API を使用
        response = client.responses.create(
            model="gpt-4o-mini",
            input=prompt,
            store=False
        )

        message_content = response.output_text

        if message_content is None or message_content.strip() == "":
            return "お疲れさまでした。少し休んでくださいね。"

        return message_content.strip()
    except Exception as e:
        print(f"メッセージ生成エラー: {e}")
        return "あなたの気持ちを分析しました。今日も一日お疲れさまでした。"


def get_emotion_advice(user_input, emotions, pad_values):
    """
    【新機能】感情別AIアドバイザー - Responses APIで具体的なアドバイスを提供

    Args:
        user_input (str): ユーザーの入力文
        emotions (dict): 6つの感情スコア
        pad_values (dict): PAD値

    Returns:
        tuple: (response_id, advice_text)
    """
    global current_response_id

    # 感情スコアを整形
    emotions_text = "\n".join([f"- {key}: {value}%" for key, value in emotions.items()])

    # 最も高い感情を特定
    top_emotion = max(emotions.items(), key=lambda x: x[1])

    prompt = f"""あなたは共感力の高いメンタルヘルスアドバイザーです。

【ユーザーの状況】
入力: {user_input}

【感情分析結果】
{emotions_text}

最も強い感情: {top_emotion[0]} ({top_emotion[1]}%)
PAD値: pleasure={pad_values['pleasure']:.2f}, arousal={pad_values['arousal']:.2f}, dominance={pad_values['dominance']:.2f}

この状況に対して、以下の形式で具体的なアドバイスを提供してください：

1. **今の気持ちの理解**（共感的に）
2. **すぐできる対処法**（3つ、具体的で実践的な方法）
3. **長期的な心がけ**（1つ、建設的な提案）

専門用語を避け、温かく寄り添う口調で書いてください。"""

    try:
        # Responses APIで会話を開始（または継続）
        if current_response_id:
            response = client.responses.create(
                model="gpt-4o-mini",
                input=prompt,
                previous_response_id=current_response_id
            )
        else:
            response = client.responses.create(
                model="gpt-4o-mini",
                input=prompt,
                store=True  # 会話履歴を保存
            )

        # response_idを保存（次回の継続対話用）
        current_response_id = response.id

        return response.id, response.output_text

    except Exception as e:
        print(f"アドバイス生成エラー: {e}")
        return None, "アドバイスの生成に失敗しました。後ほどもう一度お試しください。"


def continue_advice_conversation(user_question):
    """
    【新機能】アドバイスについての追加質問に対応（マルチターン対話）

    Args:
        user_question (str): ユーザーの追加質問

    Returns:
        str: AIの回答
    """
    global current_response_id

    if not current_response_id:
        return "まず最初に感情分析を行ってください。"

    try:
        # previous_response_idを使って会話を継続
        response = client.responses.create(
            model="gpt-4o-mini",
            input=user_question,
            previous_response_id=current_response_id
        )

        # response_idを更新
        current_response_id = response.id

        return response.output_text

    except Exception as e:
        print(f"追加質問の処理エラー: {e}")
        return "回答の生成に失敗しました。"


def main():
    """メイン処理"""
    print("=" * 60)
    print("  📔 今日の気持ち日記")
    print("=" * 60)
    print()

    while True:
        # ユーザー入力
        user_input = input("今日の出来事を1行で教えてください：\n> ")

        if not user_input.strip():
            print("何か入力してください。")
            continue

        print("\n分析中...\n")

        # PAD分析（Responses API使用）
        pad_values = analyze_emotion_pad(user_input)

        # 6感情スコア計算
        emotions = calculate_emotions(pad_values)

        # 結果表示
        print("【感情スコア】")
        emotion_icons = {
            "喜び　": "😊",
            "怒り　": "😠",
            "恐れ　": "😨",
            "悲しみ": "😢",
            "安心　": "😌",
            "驚き　": "😲"
        }

        for emotion_name in ["喜び　", "怒り　", "恐れ　", "悲しみ", "安心　", "驚き　"]:
            icon = emotion_icons[emotion_name]
            score = emotions[emotion_name]
            bar = draw_progress_bar(score)
            padding = " " * (4 - len(emotion_name))
            print(f"{icon} {emotion_name}{padding} {bar} {score:3d}%")

        # AIメッセージ生成（Responses API使用）
        print("\n【AIからのメッセージ】")
        message = generate_message(emotions)
        print(message)
        print()

        # 【新機能】AIアドバイザー機能（自動実行）
        print("=" * 60)
        print("【AIからの具体的なアドバイス】")
        print("=" * 60)

        response_id, advice = get_emotion_advice(user_input, emotions, pad_values)
        print(advice)
        print()

        # マルチターン対話ループ
        while True:
            follow_up = input("\n💬 追加で質問はありますか？（終了する場合は Enter）\n> ")

            if not follow_up.strip():
                break

            print("\n回答中...\n")
            answer = continue_advice_conversation(follow_up)
            print(answer)
            print()

        print()
        # 継続確認
        continue_input = input("もう一度分析しますか？ (y/n): ")
        if continue_input.lower() != 'y':
            print("\nありがとうございました！またお会いしましょう 😊")
            break
        print()


# APIキーが設定されているか確認
try:
    test_key = userdata.get('openai_api_key2')
    if test_key:
        main()
    else:
        print("エラー: APIキーが設定されていません。")
        print("Google Colab Secretsに 'openai_api_key' を設定してください。")
except Exception as e:
    print(f"エラー: {e}")
    print("Google Colab Secretsに 'openai_api_key' を設定してください。")

  📔 今日の気持ち日記



KeyboardInterrupt: Interrupted by user

In [ ]:
import os
import json
import sys
from openai import OpenAI
from google.colab import userdata

# 文字コード対応（Windows/Mac/Linux）
if sys.platform == 'win32':
    sys.stdout.reconfigure(encoding='utf-8')
elif sys.platform == 'darwin':
    sys.stdout.reconfigure(encoding='utf-8')

# Google Colab Secretsから環境変数に設定
os.environ["openai_api_key"] = userdata.get('openai_api_key2')

# OpenAI APIキーの設定（直接userdataから取得）
client = OpenAI(api_key=userdata.get('openai_api_key2'))

# Responses APIのresponse_idを保持するグローバル変数
current_response_id = None
music_response_id = None  # 音楽提案用のresponse_id


def analyze_emotion_pad(text):
    """
    Responses APIを使ってPADモデルで感情分析を行う

    Args:
        text (str): 分析対象のテキスト

    Returns:
        dict: PAD値 {"pleasure": float, "arousal": float, "dominance": float}
    """
    prompt = f"""あなたは感情分析の専門家です。以下の文章をPADモデル(Pleasure-Arousal-Dominance model)で感情分析してください。

文章：{text}

必ず以下のJSON形式**のみ**で回答してください（他の文章は一切含めないでください）：
{{
  "pleasure": 0.0,
  "arousal": 0.0,
  "dominance": 0.0
}}

各値は-1.0から1.0の範囲で指定してください。
- pleasure: 快-不快の度合い（正の値=快、負の値=不快）
- arousal: 興奮-沈静の度合い（正の値=興奮、負の値=沈静）
- dominance: 支配-服従の度合い（正の値=支配的、負の値=服従的）

JSON形式のみで回答してください。"""

    try:
        # Responses API を使用
        response = client.responses.create(
            model="gpt-4o-mini",  # 利用可能なモデル名に変更
            input=prompt,
            store=False  # 会話履歴を保存しない
        )

        # output_textから結果を取得してJSONパース
        output = response.output_text.strip()

        # JSONブロックから抽出（```json ``` で囲まれている場合に対応）
        if "```json" in output:
            output = output.split("```json")[1].split("```")[0].strip()
        elif "```" in output:
            output = output.split("```")[1].split("```")[0].strip()

        pad_values = json.loads(output)
        return pad_values
    except json.JSONDecodeError as e:
        print(f"JSON解析エラー: {e}")
        print(f"受信したテキスト: {output}")
        return {"pleasure": 0.0, "arousal": 0.0, "dominance": 0.0}
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return {"pleasure": 0.0, "arousal": 0.0, "dominance": 0.0}


def calculate_emotions(pad):
    """
    PAD値から6つの基本感情スコアを計算する（改善版）

    Args:
        pad (dict): PAD値

    Returns:
        dict: 6つの感情スコア（0-100%）
    """
    p = pad["pleasure"]
    a = pad["arousal"]
    d = pad["dominance"]

    # PAD値を0-1の範囲に正規化
    def normalize(value):
        return (value + 1.0) / 2.0

    emotions = {}

    # 喜び: 快 × 興奮 × 支配
    if p > 0 and a > 0 and d > 0:
        emotions["喜び　"] = int(normalize(p) * normalize(a) * normalize(d) * 100)
    else:
        emotions["喜び　"] = 0

    # 怒り: 不快 × 興奮 × 支配
    if p < 0 and a > 0 and d > 0:
        emotions["怒り　"] = int(normalize(-p) * normalize(a) * normalize(d) * 100)
    else:
        emotions["怒り　"] = 0

    # 恐れ: 不快 × 興奮 × 服従
    if p < 0 and a > 0 and d < 0:
        emotions["恐れ　"] = int(normalize(-p) * normalize(a) * normalize(-d) * 100)
    else:
        emotions["恐れ　"] = 0

    # 悲しみ: 不快 × 沈静
    if p < 0 and a < 0:
        emotions["悲しみ"] = int(normalize(-p) * normalize(-a) * 100)
    else:
        emotions["悲しみ"] = 0

    # 安心: 快 × 沈静
    if p > 0 and a < 0:
        emotions["安心　"] = int(normalize(p) * normalize(-a) * 100)
    else:
        emotions["安心　"] = 0

    # 驚き: 興奮の絶対値
    emotions["驚き　"] = int(normalize(abs(a)) * 100)

    return emotions


def draw_progress_bar(percentage, width=20):
    """
    プログレスバーを描画する

    Args:
        percentage (int): パーセンテージ（0-100）
        width (int): バーの幅

    Returns:
        str: プログレスバーの文字列
    """
    filled = int(width * percentage / 100)
    bar = "█" * filled + "░" * (width - filled)
    return bar


def generate_message(emotions):
    """
    Responses APIを使って感情スコアから親しみやすいメッセージを生成する

    Args:
        emotions (dict): 感情スコア

    Returns:
        str: メッセージ
    """
    emotions_text = "\n".join([f"- {key}: {value}%" for key, value in emotions.items()])

    prompt = f"""以下の感情分析結果を見て、専門用語を使わず、やさしく親しみやすい言葉でユーザーに説明してください。

感情スコア：
{emotions_text}

以下の条件を守ってください：
- 100文字程度で簡潔に
- 共感的で温かい口調
- 専門用語は使わない
- ユーザーの気持ちに寄り添う表現"""

    try:
        # Responses API を使用
        response = client.responses.create(
            model="gpt-4o-mini",
            input=prompt,
            store=False
        )

        message_content = response.output_text

        if message_content is None or message_content.strip() == "":
            return "お疲れさまでした。少し休んでくださいね。"

        return message_content.strip()
    except Exception as e:
        print(f"メッセージ生成エラー: {e}")
        return "あなたの気持ちを分析しました。今日も一日お疲れさまでした。"


def get_emotion_advice(user_input, emotions, pad_values):
    """
    【新機能】感情別AIアドバイザー - Responses APIで具体的なアドバイスを提供

    Args:
        user_input (str): ユーザーの入力文
        emotions (dict): 6つの感情スコア
        pad_values (dict): PAD値

    Returns:
        tuple: (response_id, advice_text)
    """
    global current_response_id

    # 感情スコアを整形
    emotions_text = "\n".join([f"- {key}: {value}%" for key, value in emotions.items()])

    # 最も高い感情を特定
    top_emotion = max(emotions.items(), key=lambda x: x[1])

    prompt = f"""あなたは共感力の高いメンタルヘルスアドバイザーです。

【ユーザーの状況】
入力: {user_input}

【感情分析結果】
{emotions_text}

最も強い感情: {top_emotion[0]} ({top_emotion[1]}%)
PAD値: pleasure={pad_values['pleasure']:.2f}, arousal={pad_values['arousal']:.2f}, dominance={pad_values['dominance']:.2f}

この状況に対して、以下の形式で具体的なアドバイスを提供してください：

1. **今の気持ちの理解**（共感的に）
2. **すぐできる対処法**（3つ、具体的で実践的な方法）
3. **長期的な心がけ**（1つ、建設的な提案）

専門用語を避け、温かく寄り添う口調で書いてください。"""

    try:
        # Responses APIで会話を開始（または継続）
        if current_response_id:
            response = client.responses.create(
                model="gpt-4o-mini",
                input=prompt,
                previous_response_id=current_response_id
            )
        else:
            response = client.responses.create(
                model="gpt-4o-mini",
                input=prompt,
                store=True  # 会話履歴を保存
            )

        # response_idを保存（次回の継続対話用）
        current_response_id = response.id

        return response.id, response.output_text

    except Exception as e:
        print(f"アドバイス生成エラー: {e}")
        return None, "アドバイスの生成に失敗しました。後ほどもう一度お試しください。"


def continue_advice_conversation(user_question):
    """
    【新機能】アドバイスについての追加質問に対応（マルチターン対話）

    Args:
        user_question (str): ユーザーの追加質問

    Returns:
        str: AIの回答
    """
    global current_response_id

    if not current_response_id:
        return "まず最初に感情分析を行ってください。"

    try:
        # previous_response_idを使って会話を継続
        response = client.responses.create(
            model="gpt-4o-mini",
            input=user_question,
            previous_response_id=current_response_id
        )

        # response_idを更新
        current_response_id = response.id

        return response.output_text

    except Exception as e:
        print(f"追加質問の処理エラー: {e}")
        return "回答の生成に失敗しました。"


def suggest_music_playlist(user_input, emotions, pad_values):
    """
    【新機能】感情プレイリスト提案 - 感情に合わせた音楽を推薦

    Args:
        user_input (str): ユーザーの入力文
        emotions (dict): 6つの感情スコア
        pad_values (dict): PAD値

    Returns:
        tuple: (response_id, music_recommendation_text)
    """
    global music_response_id

    # 感情スコアを整形
    emotions_text = "\n".join([f"- {key}: {value}%" for key, value in emotions.items()])

    # 最も高い感情を特定
    top_emotion = max(emotions.items(), key=lambda x: x[1])

    prompt = f"""あなたは音楽療法の専門家です。

【ユーザーの感情状態】
入力: {user_input}
感情スコア:
{emotions_text}

最も強い感情: {top_emotion[0]} ({top_emotion[1]}%)
PAD値: Pleasure={pad_values['pleasure']:.2f}, Arousal={pad_values['arousal']:.2f}, Dominance={pad_values['dominance']:.2f}

この状態に最適な音楽を3つ推薦してください。

【推薦基準】
1. 感情の調整効果（リラックス/活性化）
2. 聴きやすさと親しみやすさ
3. プレイリストの見つけやすさ

【出力形式】
以下の形式で提案してください：

現在の感情: [最も強い感情とその特徴]

【おすすめジャンル Top 3】
1. 🎵 [ジャンル名]
   理由: [なぜこの音楽が適しているか]

2. 🎵 [ジャンル名]
   理由: [なぜこの音楽が適しているか]

3. 🎵 [ジャンル名]
   理由: [なぜこの音楽が適しているか]

【音楽の特徴】
テンポ: [BPMと速さの説明]
音色: [音の特徴]
リズム: [リズムの特徴]

【プレイリスト検索キーワード】
Spotify: [検索キーワード3つ]
YouTube: [検索キーワード3つ]
Apple Music: [検索キーワード3つ]

【聴き方のアドバイス】
[具体的な聴き方の提案]

専門用語は避け、音楽に詳しくない人にもわかる表現で書いてください。"""

    try:
        # Responses APIで音楽提案を生成
        response = client.responses.create(
            model="gpt-4o-mini",
            input=prompt,
            store=True  # 会話履歴を保存
        )

        # response_idを保存（次回の継続対話用）
        music_response_id = response.id

        return response.id, response.output_text

    except Exception as e:
        print(f"音楽提案生成エラー: {e}")
        return None, "音楽提案の生成に失敗しました。後ほどもう一度お試しください。"


def continue_music_conversation(user_question):
    """
    【新機能】音楽についての追加質問に対応（マルチターン対話）

    Args:
        user_question (str): ユーザーの追加質問

    Returns:
        str: AIの回答
    """
    global music_response_id

    if not music_response_id:
        return "まず最初に音楽提案を受け取ってください。"

    try:
        # previous_response_idを使って会話を継続
        response = client.responses.create(
            model="gpt-4o-mini",
            input=user_question,
            previous_response_id=music_response_id
        )

        # response_idを更新
        music_response_id = response.id

        return response.output_text

    except Exception as e:
        print(f"音楽についての追加質問の処理エラー: {e}")
        return "回答の生成に失敗しました。"


def main():
    """メイン処理"""
    print("=" * 60)
    print("  📔 今日の気持ち日記")
    print("=" * 60)
    print()

    while True:
        # ユーザー入力
        user_input = input("今日の出来事を1行で教えてください：\n> ")

        if not user_input.strip():
            print("何か入力してください。")
            continue

        print("\n分析中...\n")

        # PAD分析（Responses API使用）
        pad_values = analyze_emotion_pad(user_input)

        # 6感情スコア計算
        emotions = calculate_emotions(pad_values)

        # 結果表示
        print("【感情スコア】")
        emotion_icons = {
            "喜び　": "😊",
            "怒り　": "😠",
            "恐れ　": "😨",
            "悲しみ": "😢",
            "安心　": "😌",
            "驚き　": "😲"
        }

        for emotion_name in ["喜び　", "怒り　", "恐れ　", "悲しみ", "安心　", "驚き　"]:
            icon = emotion_icons[emotion_name]
            score = emotions[emotion_name]
            bar = draw_progress_bar(score)
            padding = " " * (4 - len(emotion_name))
            print(f"{icon} {emotion_name}{padding} {bar} {score:3d}%")

        # AIメッセージ生成（Responses API使用）
        print("\n【AIからのメッセージ】")
        message = generate_message(emotions)
        print(message)
        print()

        # 【新機能】AIアドバイザー機能（自動実行）
        print("=" * 60)
        print("【AIからの具体的なアドバイス】")
        print("=" * 60)

        response_id, advice = get_emotion_advice(user_input, emotions, pad_values)
        print(advice)
        print()

        # 【新機能】音楽プレイリスト提案
        print()
        print("=" * 60)
        print("🎧 今のあなたにおすすめの音楽")
        print("=" * 60)
        print()

        music_id, music_recommendation = suggest_music_playlist(user_input, emotions, pad_values)
        print(music_recommendation)
        print()
        print("💡 ヒント: 音楽配信サービスで上記キーワードを検索してください")
        print()

        print()
        # 継続確認
        continue_input = input("もう一度分析しますか？ (y/n): ")
        if continue_input.lower() != 'y':
            print("\nありがとうございました！またお会いしましょう 😊")
            break
        print()


# APIキーが設定されているか確認
try:
    test_key = userdata.get('openai_api_key2')
    if test_key:
        main()
    else:
        print("エラー: APIキーが設定されていません。")
        print("Google Colab Secretsに 'openai_api_key' を設定してください。")
except Exception as e:
    print(f"エラー: {e}")
    print("Google Colab Secretsに 'openai_api_key' を設定してください。")

  📔 今日の気持ち日記

今日の出来事を1行で教えてください：
> 家でゆっくり過ごせて、心が落ち着いた

分析中...

【感情スコア】
😊 喜び　  ░░░░░░░░░░░░░░░░░░░░   0%
😠 怒り　  ░░░░░░░░░░░░░░░░░░░░   0%
😨 恐れ　  ░░░░░░░░░░░░░░░░░░░░   0%
😢 悲しみ  ░░░░░░░░░░░░░░░░░░░░   0%
😌 安心　  ██████████░░░░░░░░░░  54%
😲 驚き　  ████████████░░░░░░░░  60%

【AIからのメッセージ】
あなたの気持ちを見てみると、安心感や驚きが大きいよ。何か良いことがあったのかもしれないね。その気持ち、とても大切だよ。あなたがゆったりできる瞬間を大事にしてね。

【AIからの具体的なアドバイス】
もちろんです。あなたの今の気持ちについて、一緒に考えてみましょう。

1. **今の気持ちの理解**  
   家でゆっくり過ごし、心が落ち着いたというのは、とても素晴らしい状態ですね。「安心」と「驚き」が主な感情として現れています。この状況は、普段の忙しさから解放され、自分の時間を持てる喜びを感じているのかもしれません。そんな瞬間は、心のリフレッシュにもつながります。

2. **すぐできる対処法**  
   - **リフレッシュタイムを設ける**: 自分の好きな趣味やアクティビティに没頭してみてください。読書や音楽鑑賞、アートなど、自分に合ったリラックス法でさらに心を軽くすることができます。
   - **深呼吸をする**: ゆっくりと深呼吸をすることで、心身の緊張がほぐれます。数回深い呼吸をするだけでも、より安らかな気分になれるかもしれませんよ。
   - **感謝のリストを作る**: 今の心地よい環境や、感じることができる安心感について考えて、感謝できることを書き出してみましょう。気持ちがより豊かになります。

3. **長期的な心がけ**  
   自分にとっての「安心できる空間」を定期的に設けることを意識してみてください。毎週、少しずつでもこの空間を作る時間を持つことで、ストレス軽減や心のバランスを保ちやすくなります。自分だけの特別な時間を大切にして、心のリフレッシュを継続してみてくださいね。

あなた

In [ ]:
from google.colab import userdata
import os

print("=== APIキー診断 ===\n")

# 1. Secretsから取得できるか
try:
    key = userdata.get('openai_api_key2')
    print(f"✅ Secretsから取得: 成功")
    print(f"   先頭: {key[:15]}")
    print(f"   末尾: ...{key[-10:]}")
    print(f"   長さ: {len(key)} 文字")
    print(f"   スペース含む: {' ' in key}")
    print(f"   改行含む: {'\\n' in key or '\\r' in key}")
except Exception as e:
    print(f"❌ Secretsから取得: 失敗 - {e}")
    key = None

print()

# 2. 環境変数に設定されているか
env_key = os.environ.get("OPENAI_API_KEY")
if env_key:
    print(f"✅ 環境変数: 設定済み")
    print(f"   先頭: {env_key[:15]}")
else:
    print(f"⚠️  環境変数: 未設定")

print()

# 3. APIキーの形式チェック
if key:
    if key.startswith("sk-proj-"):
        print("✅ APIキー形式: 正しい (新形式)")
    elif key.startswith("sk-"):
        print("✅ APIキー形式: 正しい (旧形式)")
    else:
        print("❌ APIキー形式: 不正")

print("\n" + "="*40)

=== APIキー診断 ===

✅ Secretsから取得: 成功
   先頭: sk-proj-Bgt3Mwt
   末尾: ...O9SdGRlmQA
   長さ: 164 文字
   スペース含む: False
   改行含む: False

✅ 環境変数: 設定済み
   先頭: sk-proj-jFwRpcM

✅ APIキー形式: 正しい (新形式)



In [ ]:
from openai import OpenAI
from google.colab import userdata  # ← 追加
import os

# 環境変数に設定（念のため）
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key2')

# または直接userdataから取得
client = OpenAI(api_key=userdata.get('openai_api_key2'))

print("=== OpenAI API接続テスト ===\n")

# テスト1: Chat Completions API（標準API）
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "こんにちは"}]
    )
    print("✅ Chat Completions API: 動作OK")
    print(f"   応答: {response.choices[0].message.content}\n")
except Exception as e:
    print(f"❌ Chat Completions API: エラー")
    print(f"   {e}\n")

# テスト2: Responses API
try:
    response = client.responses.create(
        model="gpt-4o-mini",
        input="こんにちは"
    )
    print("✅ Responses API: 動作OK")
    print(f"   応答: {response.output_text}")
except Exception as e:
    print(f"❌ Responses API: エラー")
    print(f"   {e}")

=== OpenAI API接続テスト ===

✅ Chat Completions API: 動作OK
   応答: こんにちは！今日はどんなことについてお話ししましょうか？

✅ Responses API: 動作OK
   応答: こんにちは！今日はどんなことをお話ししませんか？
